#목표


#데이터 살펴보기


In [1]:
#프로젝트를 진행함에 필요한 모듈을 불러오기
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
#파이널프로젝트_RAW_210329_210926.xlsx 마운팅 후 DataFrame으로 불러오기
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [15]:
df=pd.read_excel('/content/drive/MyDrive/미니프로젝트/파이널프로젝트_RAW_210329_210926.xlsx')
df.head()

,id,customer_id,course_id,type,state,course_title,category_title,format,completed_at,transaction_amount,coupon_title,coupon_discount_amount,sale_price,tax_free_amount,pg,method,subcategory_title,marketing_start_at
0,970634,504760.0,201435.0,ORDER,CANCELLED,올인원 패키지 : 알고리즘 기술면접 완전 정복,프로그래밍,올인원,2021-03-28 15:00:11,90500,NaN,NaN,110500.0,110500.0,NaN,VBANK,개발자 커리어,NaT
1,970650,432862.0,203178.0,ORDER,CANCELLED,초격차 패키지 : 한번에 끝내는 영상 촬영과 편집,영상/3D,올인원,2021-03-28 15:16:53,96000,NaN,NaN,116000.0,116000.0,NaN,CARD,영상 편집,2020-12-02
2,970657,72297.0,204246.0,PAYMENT,COMPLETED,시크릿코드 : 프론트엔드 실무 완성편,프로그래밍,스쿨 온라인,2021-03-28 15:21:43,171810,[10% 할인] 시크릿코드 실무 완성편,19090.0,190900.0,190900.0,NaN,CARD,NaN,2021-03-04
3,970656,72297.0,204246.0,ORDER,CANCELLED,시크릿코드 : 프론트엔드 실무 완성편,프로그래밍,스쿨 온라인,2021-03-28 15:21:43,171810,NaN,NaN,190900.0,190900.0,NaN,CARD,NaN,2021-03-04
4,970658,478028.0,201797.0,PAYMENT,COMPLETED,올인원 패키지 : 직장인 필수 스킬 3종 세트 MAX,업무 생산성,올인원,2021-03-28 15:21:54,97500,[웰컴쿠폰] 올인원 패키지 2만원 할인 쿠폰,20000.0,117500.0,117500.0,NaN,CARD,PPT/보고서,2019-11-14


| 컬럼 | 설명 |
| ---- | ----|
| (삭제) id | << 버려도 됨..??? |
| customer_id | 중복있음!! +순수 고객수 파악하려면 전처리 필요 |
| course_id | 어떤 강의 듣는지(빈 값은 폐강으로 추측) |
| type    | 해당 강좌의 현재 결제 진행상태(order는 장바구니, payment가 최종 결제) |
| state                   | type의 진행상황 |
| course_title            | 강의 제목 |
| category_title          | 강의 카테고리 제목 |
| (삭제 보류)format         | 강의 과정 분류 종류|
| completed_at            | 강의 결제 완료 시각|
| transaction_amount      | 실 결제 금액|
| coupon_title            | 쿠폰명|
| coupon_discount_amount  | 할인된 금액|
| sale_price              | 판매 금액|
| (삭제) tax_free_amount    | 면세액 |
| pg                      | 결제대행사 (빈칸은 그 외의 결제대행사)|
| method                  | 결제 방법|
| subcategory_title       | 강의 부 카테고리 제목|
| marketing_start_at      | 해당 강의가 열린날 (일단위로 기록됨)|


____
<유의할 사항>
- 쿠폰 말고도 이벤트같은 다른 조건에 의한 할인도 있다


#데이터 타입과 결측값 확인하기

>결측치 제거

In [4]:
#결측치 확인
df.isna().sum()

id                            0
customer_id                   3
course_id                  5207
type                          0
state                         0
course_title                 16
category_title               24
format                       16
completed_at                  0
transaction_amount            0
coupon_title              73602
coupon_discount_amount    73602
sale_price                 1128
tax_free_amount            1128
pg                        15075
method                    13096
subcategory_title         19829
marketing_start_at        17979
dtype: int64

In [16]:
# 카테고리가 누락된 강의들은 폐강된 강의들이므로 해당 row를 제거해준다
df = df.dropna(subset=['category_title'])

id는 단순히 자료 처리 순서이므로 삭제한다.

tax_free_amount는 보안상의 이유로 삭제한다.

In [17]:
# 'id'와'tax_free_amount' 컬럼 제거
df = df.drop(['id','tax_free_amount'],axis = 'columns')
df.head()

,customer_id,course_id,type,state,course_title,category_title,format,completed_at,transaction_amount,coupon_title,coupon_discount_amount,sale_price,pg,method,subcategory_title,marketing_start_at
0,504760.0,201435.0,ORDER,CANCELLED,올인원 패키지 : 알고리즘 기술면접 완전 정복,프로그래밍,올인원,2021-03-28 15:00:11,90500,NaN,NaN,110500.0,NaN,VBANK,개발자 커리어,NaT
1,432862.0,203178.0,ORDER,CANCELLED,초격차 패키지 : 한번에 끝내는 영상 촬영과 편집,영상/3D,올인원,2021-03-28 15:16:53,96000,NaN,NaN,116000.0,NaN,CARD,영상 편집,2020-12-02
2,72297.0,204246.0,PAYMENT,COMPLETED,시크릿코드 : 프론트엔드 실무 완성편,프로그래밍,스쿨 온라인,2021-03-28 15:21:43,171810,[10% 할인] 시크릿코드 실무 완성편,19090.0,190900.0,NaN,CARD,NaN,2021-03-04
3,72297.0,204246.0,ORDER,CANCELLED,시크릿코드 : 프론트엔드 실무 완성편,프로그래밍,스쿨 온라인,2021-03-28 15:21:43,171810,NaN,NaN,190900.0,NaN,CARD,NaN,2021-03-04
4,478028.0,201797.0,PAYMENT,COMPLETED,올인원 패키지 : 직장인 필수 스킬 3종 세트 MAX,업무 생산성,올인원,2021-03-28 15:21:54,97500,[웰컴쿠폰] 올인원 패키지 2만원 할인 쿠폰,20000.0,117500.0,NaN,CARD,PPT/보고서,2019-11-14


>결제 완료한 사람 중 환불하지 않은 사람만 판별 

In [48]:
# 결제가 완료된 사람의 데이터
df_pc = df[(df['type'] == 'PAYMENT') & (df['state'] == 'COMPLETED')]
df_pc

,customer_id,course_id,type,state,course_title,category_title,format,completed_at,transaction_amount,coupon_title,coupon_discount_amount,sale_price,pg,method,subcategory_title,marketing_start_at
2,72297.0,204246.0,PAYMENT,COMPLETED,시크릿코드 : 프론트엔드 실무 완성편,프로그래밍,스쿨 온라인,2021-03-28 15:21:43,171810,[10% 할인] 시크릿코드 실무 완성편,19090.0,190900.0,NaN,CARD,NaN,2021-03-04
4,478028.0,201797.0,PAYMENT,COMPLETED,올인원 패키지 : 직장인 필수 스킬 3종 세트 MAX,업무 생산성,올인원,2021-03-28 15:21:54,97500,[웰컴쿠폰] 올인원 패키지 2만원 할인 쿠폰,20000.0,117500.0,NaN,CARD,PPT/보고서,2019-11-14
8,478033.0,203178.0,PAYMENT,COMPLETED,초격차 패키지 : 한번에 끝내는 영상 촬영과 편집,영상/3D,올인원,2021-03-28 15:26:47,96000,[웰컴쿠폰] 올인원 패키지 2만원 할인 쿠폰,20000.0,116000.0,NaN,CARD,영상 편집,2020-12-02
10,469860.0,203644.0,PAYMENT,COMPLETED,올인원 패키지 : 권오상의 금융 아카데미 - 권오상의 재무제표 분석 기초(평생소장),부동산/금융,올인원,2021-03-28 15:27:17,263000,NaN,NaN,263000.0,NaN,CARD,NaN,2020-11-27
14,478035.0,203720.0,PAYMENT,COMPLETED,초격차 패키지 : 한 번에 끝내는 프론트엔드 개발,프로그래밍,올인원,2021-03-28 15:31:01,91000,[웰컴쿠폰] 올인원 패키지 2만원 할인 쿠폰,20000.0,111000.0,NaN,CARD,프론트엔드 개발,2021-02-02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
105411,669238.0,204286.0,PAYMENT,COMPLETED,초격차 패키지 : 한번에 끝내는 3D 디자인 툴,디자인,올인원,2021-09-25 13:39:26,212000,[미래투자금] 온라인 강의 2만원 할인쿠폰,20000.0,232000.0,INICIS,CARD,디자인툴,2021-04-23
105412,384003.0,203178.0,PAYMENT,COMPLETED,초격차 패키지 : 한번에 끝내는 영상 촬영과 편집,영상/3D,올인원,2021-09-25 13:46:29,149000,NaN,NaN,149000.0,KAKAO,CARD,영상 편집,2020-12-02
105413,670891.0,203935.0,PAYMENT,COMPLETED,초격차 패키지 : 한 번에 끝내는 파이썬 웹 개발,프로그래밍,올인원,2021-09-25 14:01:26,159000,[미래투자금] 온라인 강의 2만원 할인쿠폰,20000.0,179000.0,INICIS,CARD,백엔드 개발,2021-05-31
105416,640697.0,205365.0,PAYMENT,COMPLETED,초격차 패키지 : 한 번에 끝내는 유튜브 크리에이터 되기,영상/3D,올인원,2021-09-25 14:17:41,96000,[WELCOME] 온라인 강의 2만원 할인쿠폰,20000.0,116000.0,INICIS,VBANK,영상 편집,2021-06-15


In [49]:
# 환불이 완료된 사람의 데이터
df_rc =df[(df['type'] == 'REFUND') & (df['state'] == 'COMPLETED')]
df_rc

,customer_id,course_id,type,state,course_title,category_title,format,completed_at,transaction_amount,coupon_title,coupon_discount_amount,sale_price,pg,method,subcategory_title,marketing_start_at
651,444760.0,204122.0,REFUND,COMPLETED,퍼포먼스 마케터 되기 온라인 완주반 2기,마케팅,온라인 완주반,2021-03-29 09:24:43,-147000,NaN,NaN,404000.0,NaN,VBANK,NaN,NaT
652,472666.0,203178.0,REFUND,COMPLETED,초격차 패키지 : 한번에 끝내는 영상 촬영과 편집,영상/3D,올인원,2021-03-29 09:26:13,-96000,NaN,NaN,116000.0,NaN,CARD,영상 편집,2020-12-02
666,447177.0,203989.0,REFUND,COMPLETED,올인원 패키지 : 소금툰과 함께 캐릭터 드로잉/굿즈/임티/컷툰,디자인,올인원,2021-03-29 09:40:54,-79000,NaN,NaN,89000.0,NaN,CARD,일러스트,2021-02-19
672,373915.0,204264.0,REFUND,COMPLETED,데이터분석 프로젝트+자격증 온라인 완주반 3기,데이터사이언스,온라인 완주반,2021-03-29 09:44:01,-340000,NaN,NaN,340000.0,NaN,VBANK,NaN,2021-03-08
675,456086.0,NaN,REFUND,COMPLETED,프로모션 :연봉 앞자리가 바뀌는 프론트엔드 한끝세트 (3월 프리패스),프로그래밍,올인원,2021-03-29 09:47:17,-299000,NaN,NaN,299000.0,NaN,CARD,NaN,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
105169,600073.0,206065.0,REFUND,COMPLETED,(프로모션 7/16-7/29) 초격차 패키지 : 한 번에 끝내는 Spring 완.전.판,프로그래밍,올인원,2021-09-24 14:34:50,-149000,NaN,NaN,179000.0,NAVER,CARD,백엔드 개발,2021-07-15
105175,645911.0,NaN,REFUND,COMPLETED,프로모션 : 콘텐츠 마케터 맥북 air 패스(9월 현물 프리패스),마케팅,올인원,2021-09-24 14:41:35,-1400000,NaN,NaN,1400000.0,INICIS,CARD,NaN,NaT
105205,506356.0,206720.0,REFUND,COMPLETED,용호수의 돈 버는 실전 영상 제작,영상/3D,올인원,2021-09-24 16:10:55,-259000,NaN,NaN,259000.0,NAVER,CARD,영상 편집,2021-08-17
105245,604077.0,205569.0,REFUND,COMPLETED,올인원 패키지 : 공여사와 무작정 풀어보는 엑셀 실무,업무 생산성,올인원,2021-09-24 17:09:30,-109000,NaN,NaN,129000.0,KAKAO,POINT,엑셀/VBA,2021-08-17


In [50]:
# 결제가 완료된 사람과 환불이 완료된 사람의 데이터를 합쳐준다.
df_total = pd.merge(df_pc,df_rc,how='outer')
df_total['type'].value_counts()

PAYMENT    52899
REFUND      3821
Name: type, dtype: int64

In [54]:
# 합친 데이터 중에서 'customer_id' , 'course_title','coupon_title' 모두를 만족하는 중복값을 가장 최신값만 남기고 제거해준다.
df_total = df_total.drop_duplicates(['customer_id' , 'course_title','coupon_title'],keep = 'last')
df_total['type'].value_counts()

PAYMENT    51392
REFUND      3767
Name: type, dtype: int64

In [55]:
# 중복값을 제거한 df_total 데이터 중 type이 payment면 패캠 이용자들의 순 구매 내역,
# type이 refund면 패캠 이용자들의 순 환불 내역이다.
df_pay = df_total['type'] == 'PAYMENT'
df_pay = df_total[df_pay]
df_pay['type'].value_counts()

PAYMENT    51392
Name: type, dtype: int64

In [57]:
df_refund = df_total['type'] == 'REFUND'
df_refund = df_total[df_refund]
df_refund['type'].value_counts()

REFUND    3767
Name: type, dtype: int64

>카테고리 컬럼 수정


In [ ]:
df['category_title'].value_counts().to_frame()

# 수정 할 목록
# 1. 띄어쓰기 차이로 다른 강의로 구분된 '데이터사이언스'와 '데이터 사이언스'를 합쳐준다.
# 2. 디자인 카테고리에 서브 카테고리로 속해있는 '일러스트' 카테고리를 '디자인'카테고리에 편입시켜준다.
# 3. '교육','지식콘텐츠','비즈니스','파이낸스','관리' 카테고리는 현재 폐강된 카테고리들로 삭제해준다.

In [ ]:
# 카테고리 변경
df_category = df.replace({'category_title':{'데이터 사이언스':'데이터사이언스'}})
df_category = df_category.replace({'category_title':{'일러스트':'디자인'}})
df_category['category_title'].value_counts().to_frame()

In [ ]:
# 불필요한 카테고리 삭제
df_category = df_category[df_category.category_title != '교육']
df_category = df_category[df_category.category_title != '지식콘텐츠']
df_category = df_category[df_category.category_title != '비즈니스']
df_category = df_category[df_category.category_title != '파이낸스']
df_category = df_category[df_category.category_title != '관리']

df_category['category_title'].value_counts()

In [ ]:
df_pay[(df_pay['transaction_amount'] == 0) & (df_pay['coupon_title'].str.contains('검수'))]

In [ ]:
#웰컴쿠폰 사용자 통합
df[(df['coupon_title'].str.contains('웰컴')) | (df['coupon_title'].str.contains('WELCOME'))]

In [ ]:
df_welcome_group = df_pay[df_pay['coupon_title'].str.contains('WELCOME') | df_pay['coupon_title'].str.contains('웰컴')]

In [ ]:
df_welcome_group

#데이터 분포 확인하기

할인금액

- 얼마 할인해줬을 때, 가장 많은 결제를 이끌어냈는지

- 같은 카테고리기준으로 할인 혜택을 받은 사람과 안받은 사람의 비율



In [59]:
#할인금액 분석에서 사용할 연습용 데이터 프레임 생성
df_onlypay = df_pay[['customer_id','sale_price','transaction_amount','coupon_discount_amount']]

In [60]:
df_onlypay.describe()

,customer_id,sale_price,transaction_amount,coupon_discount_amount
count,51392.000000,5.040600e+04,5.139200e+04,2.372000e+04
mean,416866.248482,1.536335e+05,1.624575e+05,5.494965e+04
std,213400.300865,2.402381e+05,8.904012e+05,6.077120e+04
min,46.000000,1.000000e+00,0.000000e+00,4.950000e+02
25%,178620.500000,1.090000e+05,8.100000e+04,2.000000e+04
50%,503789.000000,1.360000e+05,1.190000e+05,3.000000e+04
75%,577030.500000,1.710000e+05,1.590000e+05,8.900000e+04
max,672308.000000,4.460000e+07,1.134000e+08,2.000000e+06


In [61]:
#계산시 오류나지 않게 transaction_amount를 float값으로 변환
df_onlypay = df_onlypay.astype({'transaction_amount':'float'})

# 얼마 할인받는지 계산
df_onlypay['discounted_amount'] = df_onlypay['sale_price'] - df_onlypay['transaction_amount']
df_onlypay

,customer_id,sale_price,transaction_amount,coupon_discount_amount,discounted_amount
0,72297.0,190900.0,171810.0,19090.0,19090.0
1,478028.0,117500.0,97500.0,20000.0,20000.0
2,478033.0,116000.0,96000.0,20000.0,20000.0
3,469860.0,263000.0,263000.0,NaN,0.0
4,478035.0,111000.0,91000.0,20000.0,20000.0
...,...,...,...,...,...
52894,669238.0,232000.0,212000.0,20000.0,20000.0
52895,384003.0,149000.0,149000.0,NaN,0.0
52896,670891.0,179000.0,159000.0,20000.0,20000.0
52897,640697.0,116000.0,96000.0,20000.0,20000.0


In [ ]:
#NaN값 0.0으로 변경
df_onlypay['coupon_discount_amount'] = df_onlypay['coupon_discount_amount'].replace(np.NaN,0.0)
df_onlypay

In [76]:
#쿠폰 할인외에 이벤트 할인받은 데이터가 있는지 확인
only_coupon = (df_onlypay['coupon_discount_amount'] == df_onlypay['discounted_amount']) & (df_onlypay['discounted_amount'] != 0)
only_coupon.value_counts() 

#쿠폰 할인 받은 사람 : 21901 명  // 쿠폰 못 받은 사람: 29491 


False    29491
True     21901
dtype: int64

In [78]:
only_event = (df_onlypay['coupon_discount_amount'] != df_onlypay['discounted_amount']) & (df_onlypay['discounted_amount'] != 0)
only_event.value_counts() 

#이벤트 할인 받은 사람 : 4099 명  // 이벤트 할인 못 받은 사람: 47293 

False    47293
True      4099
dtype: int64

In [80]:
not_discount = (df_onlypay['coupon_discount_amount'] == df_onlypay['discounted_amount']) & (df_onlypay['coupon_discount_amount'] == 0)
not_discount .value_counts()

#할인 못받은 사람: = 25392 명
#할인 받은 사람: 21901 + 4099 = 26000 명

False    26000
True     25392
dtype: int64

In [83]:
df_onlypay['discount_type'] = pd.Series()
df_onlypay

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  """Entry point for launching an IPython kernel.


,customer_id,sale_price,transaction_amount,coupon_discount_amount,discounted_amount,discount_type
0,72297.0,190900.0,171810.0,19090.0,19090.0,NaN
1,478028.0,117500.0,97500.0,20000.0,20000.0,NaN
2,478033.0,116000.0,96000.0,20000.0,20000.0,NaN
3,469860.0,263000.0,263000.0,0.0,0.0,NaN
4,478035.0,111000.0,91000.0,20000.0,20000.0,NaN
...,...,...,...,...,...,...
52894,669238.0,232000.0,212000.0,20000.0,20000.0,NaN
52895,384003.0,149000.0,149000.0,0.0,0.0,NaN
52896,670891.0,179000.0,159000.0,20000.0,20000.0,NaN
52897,640697.0,116000.0,96000.0,20000.0,20000.0,NaN


In [84]:
df_not_discount = df_onlypay[not_discount]
df_not_discount['discount_type']='X'
df_not_discount

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,customer_id,sale_price,transaction_amount,coupon_discount_amount,discounted_amount,discount_type
3,469860.0,263000.0,263000.0,0.0,0.0,X
5,478031.0,299000.0,299000.0,0.0,0.0,X
6,455311.0,149500.0,149500.0,0.0,0.0,X
8,478051.0,118000.0,118000.0,0.0,0.0,X
9,131687.0,81000.0,81000.0,0.0,0.0,X
...,...,...,...,...,...,...
52889,585941.0,179000.0,179000.0,0.0,0.0,X
52891,670842.0,40000.0,40000.0,0.0,0.0,X
52892,120686.0,481000.0,481000.0,0.0,0.0,X
52893,545804.0,1800000.0,1800000.0,0.0,0.0,X


In [85]:
df_coupon = df_onlypay[only_coupon]
df_coupon['discount_type']='coupon'
df_coupon

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,customer_id,sale_price,transaction_amount,coupon_discount_amount,discounted_amount,discount_type
0,72297.0,190900.0,171810.0,19090.0,19090.0,coupon
1,478028.0,117500.0,97500.0,20000.0,20000.0,coupon
2,478033.0,116000.0,96000.0,20000.0,20000.0,coupon
4,478035.0,111000.0,91000.0,20000.0,20000.0,coupon
7,471759.0,99000.0,89000.0,10000.0,10000.0,coupon
...,...,...,...,...,...,...
52890,670833.0,189000.0,169000.0,20000.0,20000.0,coupon
52894,669238.0,232000.0,212000.0,20000.0,20000.0,coupon
52896,670891.0,179000.0,159000.0,20000.0,20000.0,coupon
52897,640697.0,116000.0,96000.0,20000.0,20000.0,coupon


In [86]:
df_event = df_onlypay[only_event]
df_event['discount_type']='event'
df_event

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,customer_id,sale_price,transaction_amount,coupon_discount_amount,discounted_amount,discount_type
192,458172.0,101000.0,0.0,111000.0,101000.0,event
200,464239.0,110500.0,0.0,117500.0,110500.0,event
209,440315.0,9900.0,0.0,10000.0,9900.0,event
211,440315.0,110500.0,0.0,115500.0,110500.0,event
214,464106.0,87600.0,0.0,125000.0,87600.0,event
...,...,...,...,...,...,...
52525,659116.0,168000.0,148000.0,0.0,20000.0,event
52560,672301.0,NaN,2000000.0,0.0,NaN,event
52641,613960.0,233000.0,0.0,300000.0,233000.0,event
52720,672307.0,NaN,600000.0,0.0,NaN,event


In [90]:
#할인 타입 합치기
df_onlypay2 = pd.concat([df_not_discount,df_coupon,df_event])
df_onlypay2.describe()

,customer_id,sale_price,transaction_amount,coupon_discount_amount,discounted_amount
count,51392.000000,5.040600e+04,5.139200e+04,5.139200e+04,5.040600e+04
mean,416866.248482,1.536335e+05,1.624575e+05,2.536203e+04,2.533642e+04
std,213400.300865,2.402381e+05,8.904012e+05,4.954743e+04,5.944097e+04
min,46.000000,1.000000e+00,0.000000e+00,0.000000e+00,-1.795000e+05
25%,178620.500000,1.090000e+05,8.100000e+04,0.000000e+00,0.000000e+00
50%,503789.000000,1.360000e+05,1.190000e+05,0.000000e+00,0.000000e+00
75%,577030.500000,1.710000e+05,1.590000e+05,2.000000e+04,2.000000e+04
max,672308.000000,4.460000e+07,1.134000e+08,2.000000e+06,7.800000e+06


In [ ]:
#할인 받은 금액에 따라 분류


할일:

discount_amount를 그룹 나누기 >> 그룹별 결제 횟수 시각화하기

카테고리별로 그룹 나누기 >> 할인 혜택 (쿠폰,이벤트 분류), 안받은사람 결제 비율 시각화

----

웰컴 할인쿠폰로 구매한 사람들 그룹화 >> customer_id 뽑아내서 웰컴 쿠폰 사용내역을 제외한 기록 분류 >> 할인쿠폰 유입 사람수 와 다음에 쿠폰 있/없이 구매한 사람들의 비율을 각각 구하기

결제 대행사별 그룹화 >> 일단 보류

재결제율

- 웰컴 할인쿠폰을 받은 사람 중 재결제한사람(신규 유입에 쿠폰의 영향?)

- 동일한 결제 수단으로 결제 여부

>최적의 가성비 할인율은 얼마?

판매 금액의 얼마정도를 할인했을 때 결제율이 상승하는지, 할인율을 10% 단위로 분류해서 비교

<추가 분석>

1. 장바구니에서 최종 결제까지 가기에 할인쿠폰/이벤트가 영향을 주는가?

2. 해비 유저와 라이트 유저 (평균값 기준으로 구분) 둘 중 누구에게 할인쿠폰/이벤트의 영향이 큰지 >> 두 그룹 중 어느 그룹이 매출에 큰 영향을 끼치는지